# Data Exploration

## Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [9]:

modes = ("mapping", "augmented_part", "augmented_full")
resolutions = (250, 500, 1000)
for resolution in resolutions:
    for mode in modes:
        with np.load(f'/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/processed/bern/{mode}/flipped/VTAs/{resolution}um.npz') as f:
            X = f['arr_0']
        print(f'{mode}, {resolution}')
        print(f'VTAs shape : {X.shape}')
        print(f'Voxel number in a VTA: {np.prod(X.shape[1:]):,}')
        print(f'Total number of voxels: {np.prod(X.shape):,}')


mapping, 250
VTAs shape : (660, 52, 57, 68)
Voxel number in a VTA: 201,552
Total number of voxels: 133,024,320
augmented_part, 250
VTAs shape : (3570, 52, 57, 68)
Voxel number in a VTA: 201,552
Total number of voxels: 719,540,640
augmented_full, 250
VTAs shape : (7425, 58, 66, 83)
Voxel number in a VTA: 317,724
Total number of voxels: 2,359,100,700
mapping, 500
VTAs shape : (660, 26, 28, 34)
Voxel number in a VTA: 24,752
Total number of voxels: 16,336,320
augmented_part, 500
VTAs shape : (3570, 26, 28, 34)
Voxel number in a VTA: 24,752
Total number of voxels: 88,364,640
augmented_full, 500
VTAs shape : (7425, 29, 33, 41)
Voxel number in a VTA: 39,237
Total number of voxels: 291,334,725
mapping, 1000
VTAs shape : (660, 13, 14, 16)
Voxel number in a VTA: 2,912
Total number of voxels: 1,921,920
augmented_part, 1000
VTAs shape : (3570, 13, 14, 16)
Voxel number in a VTA: 2,912
Total number of voxels: 10,395,840
augmented_full, 1000
VTAs shape : (7425, 14, 17, 21)
Voxel number in a VTA: 4,99

In [8]:
with np.load('/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/processed/augmented_interp_full/bern/flipped/VTAs/250um.npz') as f:
    X = f['arr_0']

In [9]:
print(f'X shape : {X.shape}')
print(f'Voxel number in a VTA: {np.prod(X.shape[1:]):,}')
print(f'Total number of voxels: {np.prod(X.shape):,}')

X shape : (7425, 58, 66, 83)
Voxel number in a VTA: 317,724
Total number of voxels: 2,359,100,700


In [11]:
resolutions = ('250', '500', '1000')
bbox_idx = {}
for resolution in resolutions:
    with np.load(f'/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/processed/augmented_interp_part/bern/flipped/VTAs/{resolution}um.npz') as f:
        X = f['arr_0']
        
    # minX, minY, minZ, maxX, maxY, maxZ
    bbox0, bbox1, bbox2, bbox3, bbox4, bbox5 = [], [], [], [], [], []
    for i, vta in enumerate(X):
        # X [x, y, z]
        if vta[0][:][:].flatten().sum():
            bbox0.append(i)
        if vta[-1][:][:].flatten().sum():
            bbox3.append(i)
        if vta[:][0].flatten().sum():
            bbox1.append(i)
        if vta[:][-1].flatten().sum():
            bbox4.append(i)
        if vta[:][:][0].flatten().sum():
            bbox2.append(i)
        if vta[:][:][-1].flatten().sum():
            bbox5.append(i)
    #print(bbox0, bbox1, bbox2, bbox3, bbox4, bbox5)
    bbox_buff = list(set(bbox0 + bbox1 + bbox2 + bbox3 + bbox4 + bbox5))
    print(bbox_buff)
    bbox_idx[resolution] = bbox_buff

[547, 2076, 3029]
[547, 3269, 1222, 1223, 3037, 3088, 3027, 252, 3028, 3029, 2076, 253]
[2309, 2310, 3088, 660, 661, 2073, 2074, 2075, 1818, 285, 284, 283, 1817, 2076, 547, 1208, 3267, 3268, 3269, 1222, 1223, 2894, 3025, 3026, 3027, 3028, 1493, 1494, 3029, 3284, 3035, 3036, 3037, 3053, 3054, 250, 251, 252, 253]


In [20]:

table_psm_path = f'/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/raw/psm_tables/{dataset_type}/table_psm.csv'

[547, 2076, 3029]

## N-image

In [4]:
with np.load('../data/processed/augmented/bern/VTAs/250um_n-image.npz') as f:
    n_image = f['arr_0']

n_image.shape

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/augmented/bern/VTAs/250um_n-image.npz'

In [ ]:
from ipywidgets import interactive

def plotting_n_image(depth=0):
    data = n_image[:,:,depth]
    ax = sb.heatmap(data, vmax=np.max(n_image), vmin=np.min(n_image))
    ax.set(title='N image')
    ax.set(xticklabels=[])
    ax.set(xlabel=None)
    ax.set(yticklabels=[])
    ax.set(ylabel=None)

interactive_plot = interactive(plotting_n_image, depth = (0,n_image.shape[2]-1,1))
interactive_plot

interactive(children=(IntSlider(value=0, description='depth', max=67), Output()), _dom_classes=('widget-intera…

## Chunkified N-image

In [ ]:
def chunkify(n_image, N=4):
    x, y, z = n_image.shape
    # int division
    assert x >= N and y >= N and z >= N, 'Chunks size larger than one of the data dimensions'
    x_bins = np.array_split(range(x), N)
    y_bins = np.array_split(range(y), N)
    z_bins = np.array_split(range(z), N)
    
    chunks = np.array([np.sum(n_image[x[0]:x[-1], y[0]:y[-1], z[0]:z[-1]]) for x in x_bins
    for y in y_bins for z in z_bins])
    print(chunks.shape)
    chunks = np.reshape(chunks,(N,N,N))
    print(chunks.shape, np.prod(chunks.shape))
    return chunks


In [ ]:
chunks = chunkify(n_image, 20)
#print(chunks)

(8000,)
(20, 20, 20) 8000


In [ ]:
def plotting_chunks(depth=0):
    data = chunks[:,:,depth]
    ax = sb.heatmap(data, vmin=np.min(chunks), vmax=np.max(chunks))
    ax.set(title='N image')
    ax.set(xticklabels=[])
    ax.set(xlabel=None)
    ax.set(yticklabels=[])
    ax.set(ylabel=None)

interactive_plot = interactive(plotting_chunks, depth = (0,chunks.shape[2]-1,1))
interactive_plot

interactive(children=(IntSlider(value=0, description='depth', max=19), Output()), _dom_classes=('widget-intera…

## Mean-image

In [ ]:
with np.load('../data/processed/augmented/bern/VTAs/250um_mean-image.npz') as f:
    mean_image = f['arr_0']
mean_image.shape

(152, 58, 68)

In [7]:
# measures of dispersion
min = np.amin(mean_image)
max = np.amax(mean_image)
range_ = np.ptp(mean_image)
variance = np.var(mean_image)
sd = np.std(mean_image)

print("Measures of Dispersion")
print("Minimum =", min)
print("Maximum =", max)
print("Range =", range_)
print("Variance =", variance)
print("Standard Deviation =", sd)

#stats.describe(mean_image)


NameError: name 'mean_image' is not defined

In [6]:
def plotting_mean_image(depth=0):
    data = mean_image[:,:,depth]
    ax = sb.heatmap(data, vmax=np.max(mean_image), vmin=np.min(mean_image))
    ax.set(title='Mean image')
    ax.set(xticklabels=[])
    ax.set(xlabel=None)
    ax.set(yticklabels=[])
    ax.set(ylabel=None)

interactive_plot = interactive(plotting_mean_image, depth = (0,mean_image.shape[2]-1,1))
interactive_plot

NameError: name 'interactive' is not defined

Mean-image computation is outlier-sensitive.

In [ ]:

modes = ("mapping", "augmented_part", "augmented_full")
resolutions = (250, 500, 1000)
for resolution in resolutions:
    for mode in modes:
        with np.load(f'/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/processed/bern/{mode}/flipped/VTAs/{resolution}um.npz') as f:
            X = f['arr_0']
        print(f'{mode}, {resolution}')
        
